In [1]:
import sys
base_dir = "D:/Thesis/Cobra/cobra/"
if base_dir not in sys.path:
    sys.path.append(base_dir)
from os.path import join
import os
import pandas as pd
from utilities import utils
from stats_tools import vis as svis
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.dpi"] = 40
plt.style.use('ggplot')
import importlib
importlib.reload(svis)
from pydicom import dcmread

In [10]:
sif_dir = 'Y:/'
fig_dir = join(base_dir, 'figs')
table_dir = join(base_dir, 'data/tables')
dfc = utils.load_scan_csv(join(table_dir, 'neg_pos_clean.csv'))

C:\Users\klein\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.


### Directories on sif

In [11]:
df_sif_dir = utils.load_scan_csv(join(table_dir, 'series_directories.csv'))
sif_dir_dic= pd.Series(df_sif_dir.Directory.values, index=df_sif_dir.SeriesInstanceUID).to_dict()

## Get 3d T1 scan pairs

In [12]:
df_3dt1 = dfc[(dfc.Sequence=='t1') & (dfc.MRAcquisitionType=='3D') & (dfc.NumberOfSlices>=64)]
print(len(df_3dt1), len(df_3dt1.PatientID.unique()))
gb_pat_date = df_3dt1.groupby('PatientID').InstanceCreationDate.nunique()
gb_masked = gb_pat_date[gb_pat_date>1]
pat_scan_pairs = list(gb_masked.index)
print(len(pat_scan_pairs))
df_3dt1_pairs = df_3dt1[df_3dt1.PatientID.isin(pat_scan_pairs)]
print(len(df_3dt1_pairs))

35594 14005
1724
8711


In [ ]:

    n_slices = []
    counter=0
    df_pat_dirs = pd.read_csv(
        "D:\Thesis\Cobra\cobra\data\\tables\patient_directories.csv")
    dic_pat_dis = pd.Series(
    df_pat_dirs.Directory.values, index=df_pat_dirs.PatientID)\
        .to_dict()
    for index, row in data_frame.iterrows():         
        counter+=1
        if counter%1000==0:
            print(f'{counter} volumes written \n')
            print(dt.now())
        patient_id = row['PatientID']
        series_id = row['SeriesInstanceUID']
        try:
            vol_path = join(disk_path, 'Cobra', 'Data', 'dcm', 
                            dic_pat_dis[patient_id], series_id)
            n_slices = len(os.listdir(vol_path))
        except:
            n_slices = find_n_slices(patient_id, row['StudyInstanceUID'],
                                    series_id, sif_path)
        with open(txt_file_path, mode="a+") as f:
            f.write(f"{series_id}, {n_slices}\n")
    print(f"Number of scans written to {txt_file_path}: {counter}")
    return 0

In [43]:
def save_missing_tags_to_txt(sids, txt_file_path, 
    key_list=['PercentSampling', 
    'PercentPhaseFieldOfView', 
    'PixelBandwidth',
    'ReconstructionDiameter']):
    """Save missing tags to text."""
    table_dir = "D:/Thesis/Cobra/cobra/data/tables"
    df_sif_dir = utils.load_scan_csv(join(table_dir, 'series_directories.csv'))
    sif_dir_dic = pd.Series(df_sif_dir.Directory.values, index=df_sif_dir.SeriesInstanceUID).to_dict()
    column_names = ['SeriesInstanceUID'] + key_list
    column_names_str = ",".join(column_names)
    if not os.path.exists(txt_file_path):
        with open(txt_file_path, mode="w") as f:
                f.write(f"{column_names_str}\n")
    for sid in sids:
        series_dir = join(sif_dir, sif_dir_dic[sid])
        dcm_name = os.listdir(series_dir)[0]
        dcm = dcmread(join(series_dir, dcm_name))
        save_list = [sid]
        for k in key_list:
            try:
                val = str(dcm[k].value)
            except:
                val = str(None)
            save_list.append(val)
        save_string = ",".join(save_list)
        with open(txt_file_path, mode="a+") as f:
            f.write(f"{save_string}\n")

In [42]:
save_missing_tags_to_txt(df_3dt1_pairs.SeriesInstanceUID[:2], join(table_dir,'t1','missing_tags.txt'))

96.0
100.0
150.0
None
['bb9ad2bee80b5e36f6c01fac294c19ae', '96.0', '100.0', '150.0', 'None']
96.0
100.0
150.0
None
['c9820aaf0c5eb2140397b623f2ed9f9e', '96.0', '100.0', '150.0', 'None']


In [18]:

mr_path = 'D:\F\\test\\b91f4014288ed99919b6e6b1483ac8a4\\3809675063ea32c46697dd74c0b25455.dcm'
dcm = dcmread(mr_path)

In [19]:
dcm[0x2005, 0x140F][0]['MRAcquisitionPhaseEncodingStepsInPlane'].value
dcm[0x2005, 0x140F][0]['MRAcquisitionPhaseEncodingStepsOutOfPlane'].value
dcm[0x2005, 0x140F][0]['AcquisitionContrast'].value
dcm[0x2005, 0x140F][0]['PulseSequenceName'].value
dcm['PercentSampling'].value
dcm['PercentPhaseFieldOfView'].value
dcm['PixelBandwidth'].value
dcm['ReconstructionDiameter'].value
dcm['NumberOfPhaseEncodingSteps'].value

'256'